In [2]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [4]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

売上データの傾向を分析するために、いくつかの観点からデータを検討してみましょう。

### 1. カテゴリー別の売上分析
まず、各カテゴリー（食品、衣服、日用品）の売上に注目します。

- **食品**の売上:
  - 商品コードが1001から1003、1085、1086などが含まれており、比較的多くの取引があります。
  - 売上単価が高い商品（例：ザクロ、ココナッツ）と低い商品（例：りんご、バナナ）が混在しています。

- **衣服**の売上:
  - Tシャツ、ジーンズ、レインパンツ、レインシューズなどの多様な商品があり、比較的高い単価のアイテムも存在します。
  - ジーンズなどの高価格商品が売上の大部分を占める傾向があります。

- **日用品**の売上:
  - バスブラシのような単価の低い商品が含まれており、数量が多い商品の販売も見受けられます。

### 2. 時系列分析
売上日を考慮し、特定の期間における売上の変動を確認します。

- 2023年の初めには比較的購入数が多く、特に日用品や食料品に対する需要が高かった可能性があります。
- 月末や特定のイベント（例：連休、キャンペーンなど）がある日には、売上が急増する傾向があるかもしれません。

### 3. 利益分析
原価と売上単価を元に、各商品の利益を計算し、以下の観点で分析することができます。

- 利益 = (単価 - 原価) * 数量
  - この計算により、高利益商品を特定し、リソースの最適配分が図れます。
  
### 4. 売上上位商品
特に売上が高い商品を特定し、それに基づいて次の施策を考えます。

- 売上が最大の食品・衣服の商品のプロモーションを強化すると良いかもしれません。

### 5. 顧客セグメンテーション
購入頻度や数量に基づき、顧客セグメンテーションを行うことで、マーケティング施策の最適化が図れます。

### 結論
この売上データを基に、特定のカテゴリーや商品に注目したマーケティング施策の実施、季節やイベントに応じた情報提供、利益を増やすための戦略的計画が求められます。また、売上の季節的変動を理解することで、有効なプロモーション活動のタイミングを見定めることが可能です。


In [5]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0            売上データの傾向を分析するために、いくつかの観点からデータを検討してみましょう。
1                                                    
2                                  ### 1. カテゴリー別の売上分析
3                      まず、各カテゴリー（食品、衣服、日用品）の売上に注目します。
4                                                    
5                                        - **食品**の売上:
6     - 商品コードが1001から1003、1085、1086などが含まれており、比較的多くの...
7     - 売上単価が高い商品（例：ザクロ、ココナッツ）と低い商品（例：りんご、バナナ）が混在し...
8                                                    
9                                        - **衣服**の売上:
10    - Tシャツ、ジーンズ、レインパンツ、レインシューズなどの多様な商品があり、比較的高い単...
11                  - ジーンズなどの高価格商品が売上の大部分を占める傾向があります。
12                                                   
13                                      - **日用品**の売上:
14      - バスブラシのような単価の低い商品が含まれており、数量が多い商品の販売も見受けられます。
15                                                   
16                                       ### 2. 時系列分析
17                      売上日を

In [6]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [3]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
